In [3]:
import pandas as pd
import numpy as np 
import os
import sys
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
import statistics 
from statistics import mode 
import math

Goal : Given an asset A and time intervals - t1 and t2, we study the following effect. 

Suppose the returns of asset A resampled over t1 is above a certain threshold which we call the pump threshold. We then ask what effect that might have over the interval - t2 starting from the end of interval t1. 

### Loading df

In [4]:
def load_df(path):
    df = pd.read_csv(path)
    df.columns = ['timestamp','open','high','low','close','volume']
    df['date'] = pd.to_datetime(df['timestamp'], unit = 'ms')
    df = df.set_index(pd.DatetimeIndex(df['date']))
    df = df.drop(['date'],axis = 1)        
    df['returns'] = df['open'].pct_change()
    print('Total NaN : ' + "\n")
    print(df.isna().sum())
    return(df)

### Checking for missing values

In [5]:
def check_for_missing_timestamps(df):
    m = len(df)
    X = df['timestamp'].values
    count = 0
    missing_indices = []
    
    
    l_temp = [X[i]-X[i-1] for i in range(1,len(X))]
    interval_length = mode(l_temp)    # We assume that we more or less have all the data. 
    
    
    for i in range(1,m):
        if X[i]-X[i-1] != interval_length:
            count += 1
            error = np.timedelta64(X[i]-X[i-1],'ms')
            missing_indices.append((i,error))
    
    t = (np.array(missing_indices).sum())
    t = t.astype('int')
    t = t/60000 
    
    print('Number of instances for which we have intervals of missing data is {}.'.format(count))
    print('')
    print('Total amount of missing time in dataframe is {} minutes'.format(t))
    
    return(count,missing_indices)     

In [6]:
def resample(df,interval):
    """
    Input : df -- dataframe OHLCV with pandas datetime as index.
            interval -- string eg. 1T,1H,1D,1m 
    
    Output : df resampled. 
    
    In the resample code, T -- minute eg 5T will resample for 5 min intervals,
    H -- hours, D -- days, m -- months.
    """
    
    ohlc_dict = {
        'open':'first',
        'high':'max',
        'low':'min',
        'close':'last',
        'volume':'sum'
        }
    
    df = df.resample(interval).agg(ohlc_dict)
    
    df['returns'] = df['open'].pct_change()
    
    df = df.dropna(axis = 0)
    
    return(df)

In [7]:
def plot_daily_price_volume(df):
    df_1d = resample(df,'1D')

    area_chart = px.area(df_1d.close, title = 'Daily price')
    
    area_chart.update_xaxes(title_text = 'Date')
    area_chart.update_yaxes(title_text = 'close price', tickprefix = '$')
    area_chart.update_layout(showlegend = False)
    area_chart.show()
    
    area_chart = px.area(df_1d.volume, title = 'Daily volume')

    area_chart.update_xaxes(title_text = 'Date')
    area_chart.update_yaxes(title_text = 'volume')
    area_chart.update_layout(showlegend = False)
    area_chart.show()

In [8]:
def check_for_pump_action(df,pump_thresh):
    """
    input - 
        df : dataframe with a column for returns data.
        pump_thresh : pump threshold
        
    output -
        singular_indices : indices where the returns data is above 
                       the threshold
    """
    a = np.percentile(df['returns'],pump_thresh)
    
    m = len(df)
    singular_indices = []
    for i in range(m):
        if df['returns'][i] >= a:
            singular_indices.append(i)
    singular_date_indices = df.index[singular_indices]        
    return(singular_indices,singular_date_indices)

In [9]:
path = '../Binance/crypto_pumps/data/raw/Binance/BTC_USDT.csv'

In [10]:
btcusd = load_df(path)

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64


In [11]:
btcusd = btcusd.dropna(axis = 0)

In [12]:
check_for_missing_timestamps(btcusd)

Number of instances for which we have intervals of missing data is 22.

Total amount of missing time in dataframe is 5160.963466666667 minutes


(22,
 [(1199, numpy.timedelta64(945211,'ms')),
  (2505, numpy.timedelta64(2100000,'ms')),
  (196750, numpy.timedelta64(36060000,'ms')),
  (198250, numpy.timedelta64(6360000,'ms')),
  (207468, numpy.timedelta64(27480000,'ms')),
  (361428, numpy.timedelta64(12660000,'ms')),
  (398418, numpy.timedelta64(25260000,'ms')),
  (567918, numpy.timedelta64(21660000,'ms')),
  (659778, numpy.timedelta64(36060000,'ms')),
  (693392, numpy.timedelta64(3720000,'ms')),
  (791537, numpy.timedelta64(28860000,'ms')),
  (920657, numpy.timedelta64(8460000,'ms')),
  (920727, numpy.timedelta64(240000,'ms')),
  (937794, numpy.timedelta64(7260000,'ms')),
  (1047114, numpy.timedelta64(3660000,'ms')),
  (1062030, numpy.timedelta64(21300000,'ms')),
  (1081702, numpy.timedelta64(7740000,'ms')),
  (1156012, numpy.timedelta64(9060000,'ms')),
  (1248022, numpy.timedelta64(12660000,'ms')),
  (1471252, numpy.timedelta64(3660000,'ms')),
  (1501922, numpy.timedelta64(13860000,'ms')),
  (1506722, numpy.timedelta64(3660000,'

In [13]:
btcusd_5t = resample(btcusd,'5T')

In [14]:
singular_indices, singular_date_indices = check_for_pump_action(btcusd_5t,95)

In [15]:
btcusd_5t.loc[singular_date_indices[0],:]

open       8350.000000
high       8369.990000
low        8250.000000
close      8277.000000
volume      259.052820
returns       0.005539
Name: 2018-02-09 10:05:00, dtype: float64

In [16]:
btcusd_5t.iloc[singular_indices]

,open,high,low,close,volume,returns
date,,,,,,
2018-02-09 10:05:00,8350.00,8369.99,8250.00,8277.00,259.052820,0.005539
2018-02-09 10:40:00,8298.96,8310.00,8263.93,8269.00,77.434147,0.010834
2018-02-09 11:20:00,8312.00,8339.00,8278.69,8323.98,136.046677,0.004735
2018-02-09 11:35:00,8329.89,8345.00,8305.96,8340.67,93.668587,0.004703
2018-02-09 12:30:00,8231.04,8286.06,8231.02,8264.00,94.743231,0.003542
...,...,...,...,...,...,...
2021-02-08 06:15:00,38994.91,39101.19,38973.29,39070.75,439.207772,0.005268
2021-02-08 07:30:00,39118.64,39380.89,39113.32,39362.40,472.592240,0.003078
2021-02-08 07:35:00,39362.40,39391.78,39306.26,39349.99,295.290775,0.006231


In [17]:
np.percentile(btcusd_5t['returns'],95)

0.003070447191504953

In [18]:
## mapping datetime indices back and forth

m1 = len(btcusd_5t)
m2 = len(btcusd)

#for i in df1:
    

In [19]:
y = btcusd.index[4]

In [20]:
x = btcusd_5t.index[0]

In [21]:
x - y

Timedelta('-1 days +23:59:45.211000')

In [22]:
(x-y) < pd.Timedelta(1,unit = "m") and (x-y) > pd.Timedelta(0,unit = "m")

False

In [23]:
btcusd_2d = resample(btcusd,'2D')

In [24]:
y = btcusd.index[2265]

x = btcusd_2d.index[0]

In [25]:
x - y

Timedelta('-1 days +23:59:00')

In [26]:
x > y

False

In [27]:
(x-y) < pd.Timedelta(1,unit = "m") and (x-y) >= pd.Timedelta(0,unit = "m")

False

In [28]:
y

Timestamp('2018-02-11 00:01:00')

In [29]:
x

Timestamp('2018-02-11 00:00:00', freq='2D')

In [30]:
## Assume len(df1) is less than len(df2) i.e. df1 is finer than df2

def index_mapping(df1,df2):
    m1 = len(df1)
    m2 = len(df2)
    
    i = 0
    j = 0
    current = df2.index[j]
    
    index_mapping_before = {}
    index_mapping_after = {}
    
    while i < m1:
        if df1.index[i] <= current:
            i += 1
        
        if df1.index[i] > current:
            index_mapping_before[current] = df1.index[i-1]
            index_mapping_after[current] = df1.index[i]
            i += 1
            j += 1
            if j >= m2:
                break
            else: 
                current = df2.index[j]
                
    return(index_mapping_before,index_mapping_after)

In [31]:
p,q = index_mapping(btcusd,btcusd_5t)

In [32]:
def get_dict_datetime_index(df):
    l = list(df.index)
    d = {}
    for i,a in enumerate(l):
        d[a] = i
    return(d)    

In [102]:
def get_follow_up(df1,df2,singular_indices,singular_date_indices,follow_up_measure):
    
    index_mapping_before, index_mapping_after = index_mapping(df1,df2)
    mode2 = mode([df2.index[i]- df2.index[i-1] for i in range(1,100)])
    
    singular_indices_with_holes_after = []
    
    tranches = []
    
    d1 = get_dict_datetime_index(df1)
    d2 = get_dict_datetime_index(df2)
    
    for a in tqdm(singular_date_indices):
        a_ind = d2[a]
        if a_ind + 1 <= len(df2)-1:
            if df2.index[a_ind + 1] - a != mode2:
                singular_indices_with_holes_after.append(a)
            else:
                index_df = index_mapping_after[a]
                
                tranch = []            
                tmp = d1[index_df]
                i = tmp
                
                while df1.index[i] - index_df < follow_up_measure:
                    tranch.append(df1.index[i])
                    i += 1
                #print(len(tranch))    
                tranches.append(tranch)    
                
        else:
            singular_indices_with_holes_after.append[a]
    
    return(tranches,singular_indices_with_holes_after)


In [103]:
follow_up_measure = pd.Timedelta(3,unit = "m")

In [104]:
tranches,_ = get_follow_up(btcusd,btcusd_5t,singular_indices,singular_date_indices,follow_up_measure)

In [105]:
def tranch_quality_check(tranches):
    mode1 = mode([len(x) for x in tranches])
    count = 0
    for x in tranches:
        if len(x) != mode1:
            count += 1
    return(count)

In [106]:
tranch_quality_check(tranches)

0

In [122]:
def aggregate_df(df,d):
    d['date'].append(df.index[0])
    d['open'].append(df['open'][0])
    d['high'].append(max(df['high']))
    d['low'].append(min(df['low']))
    d['close'].append(df['close'][-1])
    d['volume'].append(sum(df['volume']))
    d['intra_interval_returns'].append(100*(df['close'][-1] - df['open'][0])/df['open'][0])
    return(d)

In [124]:
def create_follow_up_df(df,tranches):
    d = {}
    for x in ['date','open','high','low','close','volume','intra_interval_returns']:
        d[x] = []
        
    for t in tranches:
        e = df.loc[t]
        d = aggregate_df(e,d) 
    
    follow_up_df = pd.DataFrame(d)
    follow_up_df = follow_up_df.set_index('date')
    
    return(follow_up_df)

In [130]:
# probability that strategy would be successful (this honestly makes no sense since we are 
# after the fact information.)

len(follow_up_df[follow_up_df['intra_interval_returns'] > 0])/len(follow_up_df)

0.45760447808663574

In [125]:
follow_up_df = create_follow_up_df(btcusd,tranches)

In [128]:
len(follow_up_df[follow_up_df['intra_interval_returns'] > 0])/len(follow_up_df)

0.45760447808663574

In [46]:
btcusd.loc[tranches[0]].apply({
        'open':'first',
        'high':'max',
        'low':'min',
        'close':'last',
        'volume':'sum'
        }, axis = 0)

RecursionError: maximum recursion depth exceeded in comparison

In [38]:
type(btcusd.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [39]:
btcusd_5t

,open,high,low,close,volume,returns
date,,,,,,
2018-02-09 10:05:00,8350.00,8369.99,8250.00,8277.00,259.052820,0.005539
2018-02-09 10:10:00,8251.01,8270.00,8215.78,8250.00,160.095502,-0.011855
2018-02-09 10:15:00,8235.05,8262.00,8235.05,8259.87,114.112909,-0.001934
2018-02-09 10:20:00,8259.92,8263.00,8200.00,8214.50,172.633071,0.003020
2018-02-09 10:25:00,8211.09,8214.50,8166.00,8200.29,156.147257,-0.005912
...,...,...,...,...,...,...
2021-02-08 11:35:00,39381.73,39441.10,39333.71,39400.06,181.066286,-0.000210
2021-02-08 11:40:00,39400.06,39430.48,39350.00,39408.29,169.593536,0.000465
2021-02-08 11:45:00,39405.48,39447.00,39353.01,39356.60,273.071159,0.000138


In [40]:
btcusd[:10]

,timestamp,open,high,low,close,volume,returns
date,,,,,,,
2018-02-09 10:01:14.789,1518170474789,8304.00,8369.00,8304.0,8350.00,131.872916,0.008935
2018-02-09 10:02:14.789,1518170534789,8346.50,8380.00,8330.0,8380.00,84.544616,0.005118
2018-02-09 10:03:14.789,1518170594789,8379.00,8390.00,8348.0,8360.02,112.940304,0.003894
2018-02-09 10:04:14.789,1518170654789,8350.04,8389.00,8348.0,8348.13,88.138218,-0.003456
2018-02-09 10:05:14.789,1518170714789,8350.00,8369.99,8300.0,8338.43,74.700930,-0.000005
2018-02-09 10:06:14.789,1518170774789,8320.02,8338.43,8320.0,8320.00,59.960108,-0.003590
2018-02-09 10:07:14.789,1518170834789,8320.00,8339.96,8250.0,8272.50,81.124785,-0.000002
2018-02-09 10:08:14.789,1518170894789,8289.94,8295.00,8250.0,8283.23,21.937305,-0.003613
2018-02-09 10:09:14.789,1518170954789,8266.02,8278.63,8250.0,8277.00,21.329692,-0.002885


In [41]:
a = singular_date_indices[0]

In [42]:
list(btcusd_5t.index).index(a)

0

In [43]:
singular_date_indices

DatetimeIndex(['2018-02-09 10:05:00', '2018-02-09 10:40:00',
               '2018-02-09 11:20:00', '2018-02-09 11:35:00',
               '2018-02-09 12:30:00', '2018-02-09 12:35:00',
               '2018-02-09 13:10:00', '2018-02-09 13:35:00',
               '2018-02-09 14:10:00', '2018-02-09 14:35:00',
               ...
               '2021-02-08 02:10:00', '2021-02-08 04:05:00',
               '2021-02-08 04:10:00', '2021-02-08 05:25:00',
               '2021-02-08 05:45:00', '2021-02-08 06:15:00',
               '2021-02-08 07:30:00', '2021-02-08 07:35:00',
               '2021-02-08 09:05:00', '2021-02-08 10:10:00'],
              dtype='datetime64[ns]', name='date', length=15721, freq=None)

In [44]:
mode([btcusd.index[i]-btcusd.index[i-1] for i in range(1,10000)])

Timedelta('0 days 00:01:00')